In [6]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
12,"For me, North and South (Books I&II) is the ul...",positive
270,"Long, boring, blasphemous. Never have I been s...",negative
62,This movie is about a man who likes to blow hi...,negative
800,This movie was a fascinating look at creole cu...,positive
828,If the only sex you've ever had is with a farm...,negative


In [8]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [9]:
df = normalize_text(df)
df.head()

,review,sentiment
12,me north south book i ii ultimate tv series s ...,positive
270,long boring blasphemous never glad see ending ...,negative
62,movie man like blow gas container br br he als...,negative
800,movie fascinating look creole culture society ...,positive
828,sex ever farm animal tag line movie probably s...,negative


In [10]:
df['sentiment'].value_counts()

sentiment
negative    266
positive    234
Name: count, dtype: int64

In [11]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [12]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
12,me north south book i ii ultimate tv series s ...,1
270,long boring blasphemous never glad see ending ...,0
62,movie man like blow gas container br br he als...,0
800,movie fascinating look creole culture society ...,1
828,sex ever farm animal tag line movie probably s...,0


In [13]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/CodeBy-HP/Sentiment-Classification-Mlflow-DVC.mlflow')
dagshub.init(
    repo_owner='CodeBy-HP',
    repo_name='Sentiment-Classification-Mlflow-DVC',
    mlflow=True,
    )

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "CodeBy-HP/Sentiment-Classification-Mlflow-DVC"

Repository CodeBy-HP/Sentiment-Classification-Mlflow-DVC initialized!

<Experiment: artifact_location='mlflow-artifacts:/a5fe039de92d4087bc0aea1c7b00db47', creation_time=1765866831256, experiment_id='0', last_update_time=1765866831256, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [19]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-16 12:07:53,087 - INFO - Starting MLflow run...
2025-12-16 12:07:57,330 - INFO - Logging preprocessing parameters...
2025-12-16 12:07:58,659 - INFO - Initializing Logistic Regression model...
2025-12-16 12:07:58,662 - INFO - Fitting the model...
2025-12-16 12:07:58,770 - INFO - Model training complete.
2025-12-16 12:07:58,775 - INFO - Logging model parameters...
2025-12-16 12:07:59,202 - INFO - Making predictions...
2025-12-16 12:07:59,212 - INFO - Calculating evaluation metrics...
2025-12-16 12:07:59,288 - INFO - Logging evaluation metrics...
2025-12-16 12:08:01,128 - INFO - Saving and logging the model...
2025/12/16 12:08:38 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-12-16 12:08:38,252 - INFO - Model training and logging completed in 40.92 seconds.
2025-12-16 12:08:38,253 - INFO - Accuracy: 0.616
2025-12-16 12:08:38,254 - INFO - Precision: 0.660377358490566
2025-12-